In [16]:
import pandas as pd
# Time series data - 2020 with merged covid, gdp, democracy scores

# Load the datasets
covid_data = pd.read_csv("COVID-19 Time Series Data.csv")
gdp_data = pd.read_csv("gdp_per_capita.csv", delimiter=',', skiprows=3, on_bad_lines='warn', keep_default_na=True)
democracy_data = pd.read_csv("democracy-index-eiu.csv")

# Convert the 'Date' column in the COVID dataset to datetime format
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

# Filter the COVID data for the date range from January 22, 2020, to January 22, 2022
start_date = '2020-01-22'
end_date = '2022-01-22'
covid_filtered = covid_data[(covid_data['Date'] >= start_date) & (covid_data['Date'] <= end_date)]

# Create 'Year' and 'Month' columns from the 'Date' column
covid_filtered['Year'] = covid_filtered['Date'].dt.year
covid_filtered['Month'] = covid_filtered['Date'].dt.month

# Drop the 'Province/State' column and unnecessary columns
covid_filtered = covid_filtered.drop(columns=['Province/State', 'Confirmed', 'Recovered'])

# Rename 'Country/Region' column
covid_filtered = covid_filtered.rename(columns={'Country/Region': 'Country'})

# Melt the GDP data to have a 'Year' column
gdp_melted = gdp_data.melt(id_vars='Country Name', value_vars=['2020', '2021', '2022'],
                           var_name='Year', value_name='GDP_per_Capita')

# Convert the 'Year' column to numeric and rename 'Country Name' column
gdp_melted['Year'] = pd.to_numeric(gdp_melted['Year'])
gdp_melted = gdp_melted.rename(columns={'Country Name': 'Country'})

# Filter GDP data for the years 2020 to 2022
gdp_filtered = gdp_melted[gdp_melted['Year'].isin([2020, 2021, 2022])]

# Rename columns in the democracy dataset
democracy_data = democracy_data.rename(columns={'Entity': 'Country'})

# Filter democracy data for the years 2020 to 2022
democracy_filtered = democracy_data[democracy_data['Year'].isin([2020, 2021, 2022])]

# Merge the COVID dataset with the GDP dataset on 'Country' and 'Year'
merged_data = pd.merge(covid_filtered, gdp_filtered, how='inner', on=['Country', 'Year'])

# Merge the above result with democracy data on 'Country' and 'Year'
final_merged_data = pd.merge(merged_data, democracy_filtered, how='inner', on=['Country', 'Year'])

# Group by 'Country', 'Year', and 'Date' to aggregate the data
grouped_data = final_merged_data.groupby(['Country', 'Year', 'Date']).agg({
    'Deaths': 'sum',  # Sum deaths for each country and date
    'GDP_per_Capita': 'mean',  # Average GDP per capita
    'Democracy score': 'mean',  # Average democracy scores
}).reset_index()

# Rename columns for clarity
grouped_data = grouped_data.rename(columns={
    'total_index_context': 'Democracy_Scores',
    'classification_context': 'Democracy_Classification'
})

# Export the final merged DataFrame to a CSV file
grouped_data.to_csv('time_series_data.csv', index=False)


<ipython-input-16-db56e2733118>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_filtered['Year'] = covid_filtered['Date'].dt.year
<ipython-input-16-db56e2733118>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_filtered['Month'] = covid_filtered['Date'].dt.month


In [ ]:
# Time series data with only COVID 19 - 2020 to 2022

print(covid_filtered)
# Export the final merged DataFrame to a CSV file
grouped_data.to_csv('cleaned_covid_data.csv', index=False)


             Date      Country  Deaths  Year  Month
0      2020-01-22  Afghanistan       0  2020      1
1      2020-01-23  Afghanistan       0  2020      1
2      2020-01-24  Afghanistan       0  2020      1
3      2020-01-25  Afghanistan       0  2020      1
4      2020-01-26  Afghanistan       0  2020      1
...           ...          ...     ...   ...    ...
231655 2022-01-18     Zimbabwe    5258  2022      1
231656 2022-01-19     Zimbabwe    5266  2022      1
231657 2022-01-20     Zimbabwe    5276  2022      1
231658 2022-01-21     Zimbabwe    5288  2022      1
231659 2022-01-22     Zimbabwe    5292  2022      1

[207888 rows x 5 columns]


In [20]:
# Filter COVID dataset to include 2020, 2021, and 2022
covid_filtered_years = covid_filtered[covid_filtered['Year'].isin([2020, 2021, 2022])]

# Reshape the GDP data to have a 'Year' column and include 2020, 2021, and 2022
gdp_melted = gdp_data.melt(id_vars='Country Name', value_vars=['2020', '2021', '2022'],
                           var_name='Year', value_name='GDP_per_Capita')

# Convert the 'Year' column to numeric
gdp_melted['Year'] = pd.to_numeric(gdp_melted['Year'])

# Rename 'Country Name' column
gdp_melted = gdp_melted.rename(columns={'Country Name': 'Country'})

# Merge the COVID dataset with the GDP dataset on 'Country' and 'Year'
merged_data = pd.merge(covid_filtered_years, gdp_melted, how='inner', on=['Country', 'Year'])

# Rename columns in the democracy dataset
democracy_data = democracy_data.rename(columns={'country': 'Country', 'year': 'Year'})

# Filter democracy data for the years 2020, 2021, and 2022, and select relevant columns
democracy_filtered = democracy_data[democracy_data['Year'].isin([2020, 2021, 2022])][['Country', 'Year', 'Democracy score']]

# Ensure there are no leading spaces in 'Country' names
democracy_filtered['Country'] = democracy_filtered['Country'].str.strip()

# Merge the data with COVID, GDP, and democracy scores on 'Country' and 'Year'
merged_data2 = pd.merge(merged_data, democracy_filtered, how='inner', on=['Country', 'Year'])

# Group by 'Country' and 'Year'
grouped_data = merged_data2.groupby(['Country', 'Year']).agg({
    'Deaths': 'sum',  # sum deaths for each date in month
    'GDP_per_Capita': 'mean',  # Average GDP per capita
    'Democracy score': 'mean',  # Average democracy score
}).reset_index()

# Export the final merged DataFrame to a CSV file
grouped_data.to_csv('merged_covid_gdp_democracy_2020_2022.csv', index=False)

# Print a sample of the final dataset
print(grouped_data.head())

       Country  Year   Deaths  GDP_per_Capita  Democracy score
0  Afghanistan  2020   296447      512.055098             2.85
1  Afghanistan  2021  1778958      355.777826             0.32
2  Afghanistan  2022   162238      352.603733             0.32
3      Albania  2020    88375     5343.037704             6.08
4      Albania  2021   871276     6377.203096             6.11


In [23]:
population_data = pd.read_csv('/content/API_SP.POP.TOTL_DS2_en_csv_v2_31753.csv', delimiter=',', skiprows=4, on_bad_lines='warn', keep_default_na=True)  # Replace with the correct file name

# Filter the population dataset to include only the year 2020
population_2020 = population_data[['Country Name', '2020']]  # Assuming 'Country Name' and '2020' columns exist
population_2020 = population_2020.rename(columns={'Country Name': 'Country', '2020': 'Population_2020'})

# Ensure there are no leading spaces in 'Country' names
population_2020['Country'] = population_2020['Country'].str.strip()

# Merge the population data with the existing grouped_data on 'Country'
merged_with_population = pd.merge(grouped_data, population_2020, how='inner', on='Country')

# Calculate deaths per population (deaths per capita)
merged_with_population['Deaths_per_Population'] = merged_with_population['Deaths'] / merged_with_population['Population_2020']

# Export the final merged DataFrame to a CSV file
merged_with_population.to_csv('merged_covid_gdp_democracy_population.csv', index=False)



In [21]:
correlation = merged_with_population[['Deaths_per_Population', 'GDP_per_Capita']].corr()

# Print the correlation matrix
print("Correlation between Deaths per Population and GDP per Capita:")
print(correlation)

correlation = merged_with_population[['Deaths_per_Population', 'Democracy_Scores']].corr()

# Print the correlation matrix
print("Correlation between Deaths per Population and Democracy Scores:")
print(correlation)

Correlation between Deaths per Population and GDP per Capita:
                       Deaths_per_Population  GDP_per_Capita
Deaths_per_Population               1.000000        0.256614
GDP_per_Capita                      0.256614        1.000000
Correlation between Deaths per Population and Democracy Scores:
                       Deaths_per_Population  Democracy_Scores
Deaths_per_Population               1.000000          0.356944
Democracy_Scores                    0.356944          1.000000
